In [0]:
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, LongType, DoubleType

schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", LongType(), True),
    StructField("category_id", LongType(), True),
    StructField("category_code", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("user_id", LongType(), True),
    StructField("user_session", StringType(), True)
])

In [0]:
df_october = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",header="true",schema=schema)

df_november = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv",header="true",schema=schema)

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.ecommerce.df_november AS
SELECT * FROM df_november;

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4839465715636200>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'CREATE OR REPLACE TABLE workspace.ecommerce.df_november AS\nSELECT * FROM df_november;\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:194, in SqlMagic.sql(self, line, cell)
    188 except BaseExcepti

In [0]:
df_november.createOrReplaceTempView("df_november")

In [0]:
%sql
Describe workspace.ecommerce.df_october

col_name,data_type,comment
event_time,timestamp,null
event_type,string,null
product_id,bigint,null
category_id,bigint,null
category_code,string,null
brand,string,null
price,double,null
user_id,bigint,null
user_session,string,null


In [0]:
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, LongType, DoubleType, DateType

incoming_schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_date", DateType(), True),    # NEW column
    StructField("event_type", StringType(), True),
    StructField("product_id", LongType(), True),
    StructField("category_id", LongType(), True),
    StructField("category_code", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", DoubleType(), True),       # wider numeric type
    StructField("user_id", LongType(), True),
    StructField("user_session", StringType(), True),
])

from datetime import datetime, date
from pyspark.sql import Row

try:
    bad_row = Row(
    event_time=datetime.now(),       # Python datetime
    event_date=date.today(),         # Python date
    event_type="view",
    product_id=123,
    category_id=456,
    category_code="electronics",
    brand="BrandX",
    price=99,                        # becomes double by schema
    user_id=789,
    user_session="sess1"
    )
    print(f"{bad_row}")
    df_bad = spark.createDataFrame([bad_row], incoming_schema)
    print("DF created OK (unexpected). Schema:")
    df_bad.printSchema()

    _ = df_bad.count()
    df_bad.write.format("delta").mode("append").saveAsTable(
        "workspace.ecommerce.df_october")
    print("Write succeeded")
except Exception as e:
    print(f"schema enforcement : {e}")

Row(event_time=datetime.datetime(2026, 1, 13, 12, 36, 48, 675493), event_date=datetime.date(2026, 1, 13), event_type='view', product_id=123, category_id=456, category_code='electronics', brand='BrandX', price=99, user_id=789, user_session='sess1')
DF created OK (unexpected). Schema:
root
 |-- event_time: timestamp (nullable = true)
 |-- event_date: date (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: long (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_session: string (nullable = true)

schema enforcement : [_LEGACY_ERROR_TEMP_DELTA_0007] A schema mismatch detected when writing to the Delta table (Table ID: aca019ce-07c0-486a-bcc5-e9b40f801c45).
To enable schema migration using DataFrameWriter or DataStreamWriter, please set:
'.option("mergeSchema", "true")'.
For other operations, se

In [0]:
from datetime import datetime, date
from pyspark.sql import Row, functions as F, types as T

incoming_schema = T.StructType([
    T.StructField("event_time", T.TimestampType(), True),
    T.StructField("event_date", T.DateType(), True),    # NEW column
    T.StructField("event_type", T.StringType(), True),
    T.StructField("product_id", T.LongType(), True),
    T.StructField("category_id", T.LongType(), True),
    T.StructField("category_code", T.StringType(), True),
    T.StructField("brand", T.StringType(), True),
    T.StructField("price", T.DoubleType(), True),       # wider numeric type
    T.StructField("user_id", T.LongType(), True),
    T.StructField("user_session", T.StringType(), True),
])

bad_row = Row(
    event_time=datetime.now(),       # Python datetime
    event_date=date.today(),         # Python date
    event_type="view",
    product_id=123,
    category_id=456,
    category_code="electronics",
    brand="BrandX",
    price=99,                        # becomes double by schema
    user_id=789,
    user_session="sess1"
)

df_bad = spark.createDataFrame([bad_row], incoming_schema)



(df_bad.write
  .format("delta")
  .mode("append")
  .option("mergeSchema", "true")
  .saveAsTable("workspace.ecommerce.df_october"))

In [0]:
display(spark.read.table("workspace.ecommerce.df_october").orderBy("event_date",ascending=False).limit(10))

event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_date
2026-01-13T12:37:00.653Z,view,123,456,electronics,BrandX,99.0,789,sess1,2026-01-13
2019-10-31T00:33:05.000Z,view,26403432,2053013563651392361,null,null,109.66,521158963,1c0c8fac-12e6-44d3-b6b2-a22b16fd38e2,null
2019-10-31T00:33:07.000Z,view,42200016,2095518917320508073,null,kartal,14.41,551896549,789175b3-b3b8-449f-bfeb-32e51b1f612c,null
2019-10-31T00:33:04.000Z,view,4502748,2053013563877884791,appliances.kitchen.hob,indesit,244.77,545004802,25a87d66-ca29-34a5-61c4-22af0f949ab5,null
2019-10-31T00:33:06.000Z,view,5100572,2053013553341792533,electronics.clocks,apple,431.09,523227761,860b4189-d97f-49a5-a33a-69078a11723f,null
2019-10-31T00:33:03.000Z,view,16700256,2053013559901684381,furniture.kitchen.chair,null,13.1,513790869,ecd4159b-ae37-4ae0-9681-f18934bdbb21,null
2019-10-31T00:33:02.000Z,view,16700605,2053013559901684381,furniture.kitchen.chair,zeta,16.19,514196713,f29068b7-939c-4302-9701-068fb6391443,null
2019-10-31T00:33:07.000Z,view,1480497,2053013561092866779,computers.desktop,apple,1785.12,559310552,6602b20d-df66-470f-8b48-f44fd52e3f44,null
2019-10-31T00:33:03.000Z,view,11700326,2053013554591695207,electronics.audio.acoustic,krk,362.69,522222104,b37f4d34-f360-4134-ac9d-75b7fb3bb63f,null
2019-10-31T00:33:06.000Z,view,42300027,2095518921321874323,furniture.living_room.cabinet,null,205.67,565855403,bd5190b6-52f4-4830-a5dd-849c15f2a161,null


In [0]:
dup_events = spark.read.table("workspace.ecommerce.df_november").limit(10)

In [0]:
from datetime import datetime, date
from pyspark.sql import Row

add_row = Row(
    event_time=datetime.now(),       # Python datetime
    event_type="view",
    product_id=123,
    category_id=456,
    category_code="electronics",
    brand="BrandX",
    price=99,                        # becomes double by schema
    user_id=789,
    user_session="sess1"
)

# Add a row to the dup_events table for insert action
dup_events = dup_events.union(spark.createDataFrame([add_row]))

In [0]:
dup_events.createOrReplaceTempView("ecommerce.dup_events")

In [0]:
%sql
MERGE INTO workspace.ecommerce.df_november AS T
USING dup_events as S
ON T.event_time = S.event_time
AND T.event_type = S.event_type
AND T.User_id = S.User_id
AND T.user_session = S.user_session
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
11,10,0,1
